In [1]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
import random
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm
import os
import nltk
import spacy
import random
from spacy.util import compounding
from spacy.util import minibatch

import re
import string

from collections import defaultdict
from collections import Counter


from PIL import Image
!pip install wordcloud
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import keras
from keras.models import Sequential
from keras.initializers import Constant
from keras.layers import (LSTM, Embedding, BatchNormalization, Dense, TimeDistributed, Dropout, Bidirectional, Flatten, GlobalMaxPool1D)
from keras.preprocessing.text import Tokenizer
from keras.utils.data_utils import pad_sequences
from tensorflow.keras.layers import Embedding
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam

from sklearn.metrics import (
    precision_score, 
    recall_score, 
    f1_score, 
    classification_report,
    accuracy_score
)

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 22.2.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from sklearn.datasets import fetch_20newsgroups
train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))
print(len(train.data))
print(len(test.data))

11314
7532


In [3]:
# Printing all the categories
train.target_names 

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [4]:
# Finding frequency of each category
targets, frequency = np.unique(train.target, return_counts=True)
targets, frequency

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19]),
 array([480, 584, 591, 590, 578, 593, 585, 594, 598, 597, 600, 595, 591,
        594, 593, 599, 546, 564, 465, 377], dtype=int64))

In [5]:
targets_str = np.array(train.target_names)
print(list(zip(targets_str, frequency)))

[('alt.atheism', 480), ('comp.graphics', 584), ('comp.os.ms-windows.misc', 591), ('comp.sys.ibm.pc.hardware', 590), ('comp.sys.mac.hardware', 578), ('comp.windows.x', 593), ('misc.forsale', 585), ('rec.autos', 594), ('rec.motorcycles', 598), ('rec.sport.baseball', 597), ('rec.sport.hockey', 600), ('sci.crypt', 595), ('sci.electronics', 591), ('sci.med', 594), ('sci.space', 593), ('soc.religion.christian', 599), ('talk.politics.guns', 546), ('talk.politics.mideast', 564), ('talk.politics.misc', 465), ('talk.religion.misc', 377)]


In [6]:
train1 = pd.DataFrame({'data': train.data, 'target': train.target})
train1.head()

,data,target
0,I was wondering if anyone out there could enli...,7
1,A fair number of brave souls who upgraded thei...,4
2,"well folks, my mac plus finally gave up the gh...",4
3,\nDo you have Weitek's address/phone number? ...,1
4,"From article <C5owCB.n3p@world.std.com>, by to...",14


Formatting Test Data using the same steps as training data

In [7]:
test.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [8]:
# Finding frequency of each category
targets_test, frequency_test = np.unique(test.target, return_counts=True)
targets_test, frequency_test

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19]),
 array([319, 389, 394, 392, 385, 395, 390, 396, 398, 397, 399, 396, 393,
        396, 394, 398, 364, 376, 310, 251], dtype=int64))

In [9]:
target_test_str = np.array(test.target_names)
print(list(zip(target_test_str, frequency_test)))

[('alt.atheism', 319), ('comp.graphics', 389), ('comp.os.ms-windows.misc', 394), ('comp.sys.ibm.pc.hardware', 392), ('comp.sys.mac.hardware', 385), ('comp.windows.x', 395), ('misc.forsale', 390), ('rec.autos', 396), ('rec.motorcycles', 398), ('rec.sport.baseball', 397), ('rec.sport.hockey', 399), ('sci.crypt', 396), ('sci.electronics', 393), ('sci.med', 396), ('sci.space', 394), ('soc.religion.christian', 398), ('talk.politics.guns', 364), ('talk.politics.mideast', 376), ('talk.politics.misc', 310), ('talk.religion.misc', 251)]


In [10]:
test1 = pd.DataFrame({'data': test.data, 'target': test.target})
test1.head()

,data,target
0,I am a little confused on all of the models of...,7
1,I'm not familiar at all with the format of the...,5
2,"\nIn a word, yes.\n",0
3,\nThey were attacking the Iraqis to drive them...,17
4,\nI've just spent two solid months arguing tha...,19


Text Processing: Cleaning punctuations, Removing Stop words, Stemming

In [11]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [12]:
stop_words = stopwords.words('english')
more_stopwords = ['u', 'im', 'c']
stop_words = stop_words + more_stopwords

def remove_stopwords(text):
    text = ' '.join(word for word in text.split(' ') if word not in stop_words)
    return text


In [13]:
stemmer = nltk.SnowballStemmer("english")

def stemm_text(text):
    text = ' '.join(stemmer.stem(word) for word in text.split(' '))
    return text

In [14]:
def preprocess_data(text):
    # Clean puntuation, urls, and so on
    text = clean_text(text)
    # Remove stopwords
    text = ' '.join(word for word in text.split(' ') if word not in stop_words)
    # Stemm all the words in the sentence
    text = ' '.join(stemmer.stem(word) for word in text.split(' '))
    
    return text

In [15]:
train1['clean_d'] = train1['data'].apply(clean_text)
train1.head()

,data,target,clean_d
0,I was wondering if anyone out there could enli...,7,i was wondering if anyone out there could enli...
1,A fair number of brave souls who upgraded thei...,4,a fair number of brave souls who upgraded thei...
2,"well folks, my mac plus finally gave up the gh...",4,well folks my mac plus finally gave up the gho...
3,\nDo you have Weitek's address/phone number? ...,1,do you have weiteks addressphone number id li...
4,"From article <C5owCB.n3p@world.std.com>, by to...",14,from article by tombakerworldstdcom tom a bak...


In [16]:
train1['clean_d'] = train1['clean_d'].apply(remove_stopwords)
train1.head()

,data,target,clean_d
0,I was wondering if anyone out there could enli...,7,wondering anyone could enlighten car sawthe da...
1,A fair number of brave souls who upgraded thei...,4,fair number brave souls upgraded si clock osci...
2,"well folks, my mac plus finally gave up the gh...",4,well folks mac plus finally gave ghost weekend...
3,\nDo you have Weitek's address/phone number? ...,1,weiteks addressphone number id like get infor...
4,"From article <C5owCB.n3p@world.std.com>, by to...",14,article tombakerworldstdcom tom bakermy under...


In [17]:
train1['clean_d'] = train1['clean_d'].apply(stemm_text)
train1.head()

,data,target,clean_d
0,I was wondering if anyone out there could enli...,7,wonder anyon could enlighten car sawth day sp...
1,A fair number of brave souls who upgraded thei...,4,fair number brave soul upgrad si clock oscil h...
2,"well folks, my mac plus finally gave up the gh...",4,well folk mac plus final gave ghost weekend af...
3,\nDo you have Weitek's address/phone number? ...,1,weitek addressphon number id like get informa...
4,"From article <C5owCB.n3p@world.std.com>, by to...",14,articl tombakerworldstdcom tom bakermi unders...


In [18]:
train1['clean_d'] = train1['clean_d'].apply(preprocess_data)
train1.head()

,data,target,clean_d
0,I was wondering if anyone out there could enli...,7,wonder anyon could enlighten car sawth day sp...
1,A fair number of brave souls who upgraded thei...,4,fair number brave soul upgrad si clock oscil h...
2,"well folks, my mac plus finally gave up the gh...",4,well folk mac plus final gave ghost weekend af...
3,\nDo you have Weitek's address/phone number? ...,1,weitek addressphon number id like get informa...
4,"From article <C5owCB.n3p@world.std.com>, by to...",14,articl tombakerworldstdcom tom bakermi unders...


Performing same text processing and cleaning on test data

In [19]:
test1['clean_d'] = test1['data'].apply(clean_text)
test1.head()

,data,target,clean_d
0,I am a little confused on all of the models of...,7,i am a little confused on all of the models of...
1,I'm not familiar at all with the format of the...,5,im not familiar at all with the format of thes...
2,"\nIn a word, yes.\n",0,in a word yes
3,\nThey were attacking the Iraqis to drive them...,17,they were attacking the iraqis to drive them o...
4,\nI've just spent two solid months arguing tha...,19,ive just spent two solid months arguing that n...


In [20]:
test1['clean_d'] = test1['clean_d'].apply(remove_stopwords)
test1.head()

,data,target,clean_d
0,I am a little confused on all of the models of...,7,little confused models bonnevillesi heard le ...
1,I'm not familiar at all with the format of the...,5,familiar format xface thingies butafter seeing...
2,"\nIn a word, yes.\n",0,word yes
3,\nThey were attacking the Iraqis to drive them...,17,attacking iraqis drive kuwaita country whose c...
4,\nI've just spent two solid months arguing tha...,19,ive spent two solid months arguing thing anobj...


In [21]:
test1['clean_d'] = test1['clean_d'].apply(stemm_text)
test1.head()

,data,target,clean_d
0,I am a little confused on all of the models of...,7,littl confus model bonnevillesi heard le se l...
1,I'm not familiar at all with the format of the...,5,familiar format xface thingi butaft see folk h...
2,"\nIn a word, yes.\n",0,word yes
3,\nThey were attacking the Iraqis to drive them...,17,attack iraqi drive kuwaita countri whose citiz...
4,\nI've just spent two solid months arguing tha...,19,ive spent two solid month argu thing anobject ...


In [22]:
test1['clean_d'] = test1['clean_d'].apply(preprocess_data)
test1.head()

,data,target,clean_d
0,I am a little confused on all of the models of...,7,littl confus model bonnevillesi heard le se l...
1,I'm not familiar at all with the format of the...,5,familiar format xface thingi butaft see folk h...
2,"\nIn a word, yes.\n",0,word yes
3,\nThey were attacking the Iraqis to drive them...,17,attack iraqi drive kuwaita countri whose citiz...
4,\nI've just spent two solid months arguing tha...,19,ive spent two solid month argu thing anobject ...


In [23]:
x= train1['clean_d']
y= train1['target']

In [24]:
xte = test1['clean_d']
yte = test1['target']

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

# instantiate the vectorizer
vect = CountVectorizer()
vect.fit(x)
train2 = vect.transform(x)
test2 = vect.transform(xte)

In [26]:
tfidf_transformer = TfidfTransformer()

tfidf_transformer.fit(train2)
train3 = tfidf_transformer.transform(train2)

train3

<11314x125743 sparse matrix of type '<class 'numpy.float64'>'
	with 709980 stored elements in Compressed Sparse Row format>

In [27]:
tfidf_transformer = TfidfTransformer()

tfidf_transformer.fit(test2)
test3 = tfidf_transformer.transform(test2)

test3

<7532x125743 sparse matrix of type '<class 'numpy.float64'>'
	with 398095 stored elements in Compressed Sparse Row format>

In [28]:
texts = train1['clean_d']
target = train1['target']

In [29]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(texts)

vocab_length = len(word_tokenizer.word_index) + 1
vocab_length

125807

In [30]:
def embed(corpus): 
    return word_tokenizer.texts_to_sequences(corpus)

longest_train = max(texts, key=lambda sentence: len(word_tokenize(sentence)))
length_long_sentence = len(word_tokenize(longest_train))

train_padded_sentences = pad_sequences(
    embed(texts), 
    length_long_sentence, 
    padding='post'
)

train_padded_sentences

array([[   404,     56,     17, ...,      0,      0,      0],
       [   594,     52,   1892, ...,      0,      0,      0],
       [    26,   1053,    327, ...,      0,      0,      0],
       ...,
       [   392,   1403,   2524, ...,      0,      0,      0],
       [   507,    149, 125800, ...,      0,      0,      0],
       [  2458,   5921,   1422, ...,      0,      0,      0]])

Create a Multinomial Naive Bayes model

In [31]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

# Train the model
nb.fit(train3, y)

MultinomialNB()

In [32]:
# Make class and probability predictions
y_pred_class = nb.predict(test3)
y_pred_prob = nb.predict_proba(test3)[:, 1]

In [33]:
# calculate accuracy of class predictions
from sklearn import metrics
print(metrics.accuracy_score(yte, y_pred_class))

0.6315719596388741


In [34]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

pipe = Pipeline([('bow', CountVectorizer()), 
                 ('tfid', TfidfTransformer()),  
                 ('model', MultinomialNB())])

In [35]:
# Fit the pipeline with the data
pipe.fit(x, y)

y_pred_class = pipe.predict(xte)

print(metrics.accuracy_score(yte, y_pred_class))

0.633563462559745


In [36]:
embeddings_dictionary = dict()
embedding_dim = 100

embedding_matrix = np.zeros((vocab_length, embedding_dim))

for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector
        
embedding_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [37]:
def glove_lstm():
    model = Sequential()
    
    model.add(Embedding(
        input_dim=embedding_matrix.shape[0], 
        output_dim=embedding_matrix.shape[1], 
        weights = [embedding_matrix], 
        input_length=length_long_sentence
    ))
    
    #     model.add(Bidirectional(LSTM(
#         length_long_sentence, 
#         return_sequences = True, 
#         recurrent_dropout=0.2
#     )))
    
    model.add(GlobalMaxPool1D())
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    #model.add(Dense(length_long_sentence, activation = "relu"))
    #model.add(Dropout(0.2))
    model.add(Dense(length_long_sentence, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

model = glove_lstm()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 7058, 100)         12580700  
                                                                 
 global_max_pooling1d (Globa  (None, 100)              0         
 lMaxPooling1D)                                                  
                                                                 
 batch_normalization (BatchN  (None, 100)              400       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 7058)              712858    
                                                                 
 dropout_1 (Dropout)         (None, 7058)              0

In [38]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    train_padded_sentences, 
    target, 
    test_size=0.15
)

In [39]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [50]:
model_lstm = glove_lstm()

checkpoint = ModelCheckpoint(
    'modellstm.h5', 
    monitor = 'val_loss', 
    verbose = 1, 
    save_best_only = True
)
reduce_lr = ReduceLROnPlateau(
    monitor = 'val_loss', 
    factor = 0.2, 
    verbose = 1, 
    patience = 5,                        
    min_lr = 0.001
)
history = model_lstm.fit(
    X_train, 
    y_train, 
    epochs = 20,
    batch_size = 512,
    validation_data = (X_test, y_test),
    verbose = 1,
    callbacks = [reduce_lr, checkpoint]
)

Epoch 1/20
19/19 [==============================] - ETA: 0s - loss: 0.5602 - accuracy: 0.0493
Epoch 1: val_loss improved from inf to 0.45881, saving model to modellstm.h5
19/19 [==============================] - 3s 125ms/step - loss: 0.5602 - accuracy: 0.0493 - val_loss: 0.4588 - val_accuracy: 0.0636 - lr: 0.0010
Epoch 2/20
19/19 [==============================] - ETA: 0s - loss: 0.3693 - accuracy: 0.0495
Epoch 2: val_loss improved from 0.45881 to 0.29130, saving model to modellstm.h5
19/19 [==============================] - 2s 120ms/step - loss: 0.3693 - accuracy: 0.0495 - val_loss: 0.2913 - val_accuracy: 0.0636 - lr: 0.0010
Epoch 3/20
19/19 [==============================] - ETA: 0s - loss: 0.1995 - accuracy: 0.0495
Epoch 3: val_loss improved from 0.29130 to 0.12853, saving model to modellstm.h5
19/19 [==============================] - 3s 161ms/step - loss: 0.1995 - accuracy: 0.0495 - val_loss: 0.1285 - val_accuracy: 0.0636 - lr: 0.0010
Epoch 4/20
19/19 [=============================